# Wrythm – Extração de Dados e Dataset

## Objetivo do Notebook
Este notebook tem como foco principal:

1. Extração de previews de áudio do Spotify / MusicBrainz.
2. Extração de features musicais relevantes (MFCCs, BPM, padrões rítmicos, melódicos e tom).
3. Montagem de um dataset consolidado em formato JSON para uso futuro em classificação de gêneros.
4. Exploração inicial dos dados (EDA – análise exploratória).

---

## Etapas do Pipeline

### 1. Coleta de Áudios
- Usar Spotify API para baixar previews de faixas de diferentes gêneros.
- Armazenar arquivos de áudio em pastas organizadas por gênero (`data/previews/<genero>/`).

### 2. Extração de Features
Para cada áudio coletado, extrair:

- **MFCCs**: mean e variance para capturar timbre.
- **BPM**: batidas por minuto (tempo da música).
- **Rhythmic Cell**: histograma de ataques (onsets) dividido em bins.
- **Melodic Cell**: notas dominantes por segmento.
- **Key**: estimativa do tom da música.
- **Duration**: duração do áudio em segundos.

As features serão armazenadas junto com metadados em JSON.

### 3. Montagem do Dataset
- Consolidar os arquivos JSON em uma única pasta (`data/metadata/`).
- Garantir que cada registro contenha:
  - Identificador da faixa (Spotify ID)
  - Nome e artistas
  - Caminho do arquivo de áudio
  - Features extraídas
  - Gênero(s)

### 4. Análise Exploratória Inicial (EDA)
- Contagem de faixas por gênero.
- Distribuição de duração, BPM e key.
- Estatísticas básicas de MFCCs e padrões rítmicos.
- Verificar consistência e possíveis dados faltantes.


# Desenvolvimento:

## Configurações iniciais: Imports e pré-estabelecimentos

In [2]:
# Imports e configuração inicial

# Manipulação de arquivos e diretórios
import os
from pathlib import Path
import glob
import json
import sys


# Manipulação de dados
import pandas as pd
import numpy as np


# Áudio e extração de features
import librosa
import soundfile as sf


# Visualização
import matplotlib.pyplot as plt
import seaborn as sns


# Configurações gerais
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")


# Adiciona src ao Python Path
src_path = Path("../src").resolve()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))


# Diretórios principais
PREVIEWS_ROOT = "../data/previews"   # Onde os previews de áudio serão salvos
META_ROOT     = "../data/metadata"   # Onde os JSONs com metadados e features serão salvos

# Criar diretórios caso não existam
Path(PREVIEWS_ROOT).mkdir(parents=True, exist_ok=True)
Path(META_ROOT).mkdir(parents=True, exist_ok=True)

# Variáveis de ambiente (Spotify)
from dotenv import load_dotenv
dotenv_path = Path("../../.env").resolve()
load_dotenv(dotenv_path)  # Carrega variáveis do .env

# Testa se as credenciais estão carregadas corretamente
print("SPOTIPY_CLIENT_ID =", os.getenv("SPOTIPY_CLIENT_ID"))
print("SPOTIPY_CLIENT_SECRET =", os.getenv("SPOTIPY_CLIENT_SECRET"))

# Import dos módulos personalizados
from dataset_builder import process_track_and_write_metadata, sp

print("Configurações iniciais carregadas com sucesso!")


SPOTIPY_CLIENT_ID = None
SPOTIPY_CLIENT_SECRET = None


RuntimeError: Set SPOTIPY_CLIENT_ID and SPOTIPY_CLIENT_SECRET in env vars